In [21]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import (
    plot_gauge_grid,
     plot_bar, 
     plot_time_heatmap,
     plot_statistical_strip, plot_sunburst,plot_pie,
     plot_corr_triangle
     )
from core.s3 import S3AssetManager

In [22]:
notebook_name = "template_example"
s3 = S3AssetManager(notebook_name=notebook_name)
CORPORATE_COLORS = [
    "#1A494C", "#17877D", "#94AF92", "#F6B27A", "#F18F01",
    "#E4572E", "#6C757D", "#343A40", "#A3CED0",
]

In [23]:

np.random.seed(42)

def create_dummy_data(rows=500):
    base_date = datetime(2024, 1, 1)
    dates = [base_date + timedelta(days=x) for x in range(rows)]
    plants = ["Planta Norte", "Planta Sur"]
    areas = ["Zona Húmeda", "Zona Seca", "Empaque"]
    products = ["Producto A", "Producto B", "Producto Premium"]
    operators = ["Carlos", "Ana", "Luis", "Maria", "Jorge"]
    shifts = ["Mañana", "Tarde", "Noche"]
    reactors = ["R-101", "R-102", "R-201", "R-202"]
    
    data = []
    
    for i in range(rows):
        # Selección aleatoria básica
        date = dates[i]
        plant = np.random.choice(plants, p=[0.6, 0.4]) # Planta Norte es más grande
        
        # Lógica jerárquica simple para Sunburst
        if plant == "Planta Norte":
            area = np.random.choice(areas)
            reactor = np.random.choice(["R-101", "R-102"])
        else:
            area = np.random.choice(["Zona Seca", "Empaque"]) # Sur no tiene húmeda
            reactor = np.random.choice(["R-201", "R-202"])
            
        prod = np.random.choice(products)
        op = np.random.choice(operators)
        shift = np.random.choice(shifts)
        
        # --- 3. Métricas Numéricas (con correlaciones forzadas) ---
        
        # Temperatura: Producto A requiere más calor
        base_temp = 150 if prod == "Producto A" else 120
        temp = np.random.normal(base_temp, 5) 
        
        # Presión: Correlacionada con temperatura
        pressure = (temp * 0.5) + np.random.normal(10, 2)
        
        # Rendimiento (Yield): Baja si la temperatura es muy alta (sobrecocción)
        # El operario "Carlos" es muy bueno (bonus)
        yield_base = 95
        if temp > 155: yield_base -= 10
        if op == "Carlos": yield_base += 2
        yield_pct = min(100, max(0, np.random.normal(yield_base, 3)))
        
        # Tiempo de ciclo (segundos): Para el Heatmap de tiempo
        # Más presión = menos tiempo
        cycle_time = max(1800, 3600 - (pressure * 10) + np.random.normal(0, 100))
        
        # Impurezas (ppm): Inverso al Yield
        impurity = (100 - yield_pct) * 10 + np.random.normal(0, 5)

        data.append([
            date, plant, area, reactor, prod, op, shift, 
            temp, pressure, yield_pct, cycle_time, impurity
        ])
        
    cols = [
        "Fecha", "Planta", "Area", "Reactor", "Producto", "Operario", "Turno",
        "Temperatura_C", "Presion_Bar", "Rendimiento_Pct", "Tiempo_Ciclo_Sec", "Impurezas_ppm"
    ]
    
    df = pd.DataFrame(data, columns=cols)
    
    # Agregar una categoría para Pie chart preparada
    df["Calidad"] = pd.cut(df["Rendimiento_Pct"], bins=[0, 85, 95, 100], labels=["Baja", "Media", "Alta"])
    
    return df

In [24]:
df = create_dummy_data()
df

,Fecha,Planta,Area,Reactor,Producto,Operario,Turno,Temperatura_C,Presion_Bar,Rendimiento_Pct,Tiempo_Ciclo_Sec,Impurezas_ppm,Calidad
0,2024-01-01,Planta Norte,Zona Húmeda,R-101,Producto Premium,Jorge,Mañana,118.829233,68.946343,99.737638,2987.280046,0.276244,Alta
1,2024-01-02,Planta Norte,Zona Húmeda,R-101,Producto B,Jorge,Mañana,122.712800,72.989290,90.428372,2827.302491,92.004246,Media
2,2024-01-03,Planta Norte,Zona Seca,R-102,Producto Premium,Luis,Noche,119.877724,70.649965,96.251033,2976.746539,36.022671,Alta
3,2024-01-04,Planta Sur,Empaque,R-202,Producto B,Carlos,Tarde,119.850807,70.507472,95.093321,2792.770065,48.258015,Alta
4,2024-01-05,Planta Norte,Empaque,R-101,Producto B,Carlos,Tarde,113.674404,69.021187,100.000000,3029.152105,1.093192,Alta
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2025-05-10,Planta Norte,Zona Húmeda,R-101,Producto A,Jorge,Mañana,143.379566,81.109708,93.915102,2748.164783,68.205943,Media
496,2025-05-11,Planta Norte,Zona Húmeda,R-102,Producto A,Maria,Tarde,150.397524,85.329463,95.239270,2660.782056,46.576682,Alta
497,2025-05-12,Planta Sur,Zona Seca,R-202,Producto B,Ana,Mañana,123.801609,70.645018,86.324502,3087.017943,131.383451,Media
498,2025-05-13,Planta Norte,Zona Húmeda,R-102,Producto A,Luis,Tarde,145.684825,79.596547,92.776420,2739.606351,71.572974,Media


In [40]:
fig_gauge = plot_gauge_grid(
    df=df,
    group_col="Reactor",
    value_col="Rendimiento_Pct",
    target_ranges=(90, 100),  # Meta global entre 90 y 100
    unit="%",
    label_metric="Eficiencia",
    label_count="Lotes",
    title_prefix="Status:",
    gauge_colors=["#E53935", "#FB8C00", "#43A047"] # Rojo, Naranja, Verde
)
fig_gauge.show()
s3.save_plotly_html(fig_gauge, "gauge_grid.html")

In [41]:
fig_time = plot_time_heatmap(
    df=df,
    x_col="Producto",
    y_col="Reactor",
    value_col="Tiempo_Ciclo_Sec",
    input_unit="seconds",  # La columna está en segundos
    title="Tiempo de Ciclo Promedio (HH:MM:SS)",
    colorscale=CORPORATE_COLORS
)
fig_time.show()
s3.save_plotly_html(fig_time, "time_heatmap.html")

In [42]:
fig_bar = plot_bar(
    df=df,
    x_col="Producto",
    y_col="Temperatura_C",
    group_col="Calidad",
    title="Rendimiento Promedio por Producto y Turno",
    #y_title="Yield (%)",
    compact_mode=True,     
    show_delta=True,   
    delta_reference_group="Mañana"
)
fig_bar.show()
s3.save_plotly_html(fig_bar, "bar.html")

In [43]:
fig_strip = plot_statistical_strip(
    df=df,
    x_col="Operario",
    y_col="Impurezas_ppm",
    title="Análisis Estadístico de Impurezas por Operario",
    y_title="Impurezas (ppm)",
    show_boxplot=True,
    show_mean_ci=True
)
fig_strip.show()
s3.save_plotly_html(fig_strip, "strip.html")


In [51]:
fig_sun = plot_sunburst(
    df=df,
    path_cols=["Planta", "Area", "Reactor"], # Jerarquía
    value_col="Tiempo_Ciclo_Sec",           # Tamaño del sector = Tiempo total invertido
    metric_col="Rendimiento_Pct",           # Color = Rendimiento promedio
    title="<b>Jerarquía de Producción: Tamaño por Tiempo, Color por Rendimiento</b>",
    width=800, 
    height=500
)
fig_sun.show()
s3.save_plotly_html(fig_sun, "sunburst.html")

In [45]:
cols_num = ["Temperatura_C", "Presion_Bar", "Rendimiento_Pct", "Impurezas_ppm", "Tiempo_Ciclo_Sec"]

fig_corr = plot_corr_triangle(
    df=df,
    value_cols=cols_num,
    title="Matriz de Correlación de Proceso",
    width=800, 
    height=500
)
fig_corr.show()
s3.save_plotly_html(fig_corr, "corr.html")

In [46]:
# Preparar datos como espera la función
df_pie_prep = df["Calidad"].value_counts().reset_index()
df_pie_prep.columns = ["label", "slice_value"]
df_pie_prep["hover_html"] = df_pie_prep["label"] # Dummy hover
df_pie_prep["percent"] = df_pie_prep["slice_value"] / df_pie_prep["slice_value"].sum()

fig_pie = plot_pie(
    df_prepared=df_pie_prep,
    label_col="label",
    value_col="slice_value",
    title="Distribución de Calidad de Lotes"
)
fig_pie.show()
s3.save_plotly_html(fig_pie, "pie.html")

In [49]:
df_planta = df.groupby('Planta').agg(
    Presion_Bar=("Presion_Bar","mean"),
    Rendimiento_Pct=("Rendimiento_Pct", "mean"),
    Impurezas_ppm=("Impurezas_ppm", "mean"),
    
).reset_index()
s3.save_dataframe(df_planta, 'df_planta.csv')